In [ ]:
!pip install langchain langchain-community tiktoken pypdf rapidocr-onnxruntime
!pip install -U weaviate-client
!pip install -Uqq langchain-weaviate

In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import weaviate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from weaviate.classes.init import Auth
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


https://weaviate.io/developers/weaviate/connections/connect-cloud

In [4]:
# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url= WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

print(client.is_ready())

True


In [87]:
"""from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(
    model_name = embedding_model_name
)"""

'from langchain.embeddings import HuggingFaceEmbeddings\nembedding_model_name = "sentence-transformers/all-mpnet-base-v2"\n\nembeddings = HuggingFaceEmbeddings(\n    model_name = embedding_model_name\n)'

In [5]:
embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001",  # Google's text embedding model
        google_api_key= gemini_api_key
    )

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/

In [6]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("netherlands-wikipedia-article-text.pdf",extract_images=True)
pages = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
docs = text_splitter.split_documents(pages)

docs

[Document(metadata={'source': 'netherlands-wikipedia-article-text.pdf', 'page': 0}, page_content="Netherlands - Wikipedia Text Only, Convert to PDF\nNetherlands\n   \nNetherlands national cricket team\n   \nNetherlands national football team\n   \nNetherlands Antilles\n   \nNetherlands in World\nWar II\n   \nNetherlands Armed Forces\n   \nNetherlands Marine Corps\n   \nNetherlands women's national football team\n   \nNetherlands at the\nFIFA World Cup\n   \nNetherlands in the Eurovision Song Contest\n   \nNetherlands\n \nThe \nNetherlands\n (Dutch: \nNederland\n \n[ˈneːdərlɑnt]\n \n), informally \nHolland\n, is a\ncountry located in northwestern Europe with overseas territories in the Caribbean.\nIt is the largest of four constituent countries of the Kingdom of the Netherlands.\nThe Netherlands consists of twelve provinces; it borders Germany to the east, and\nBelgium to the south, with a North Sea coastline to the north and west. It also has\na border with France on the split island o

In [8]:

from langchain_weaviate.vectorstores import WeaviateVectorStore

In [11]:
client.collections.delete("LangChain")

In [12]:
vectorstore = WeaviateVectorStore.from_documents(
        docs,
        embeddings,
        client=client,
        index_name="LangChain"
    )
print("Vector store setup complete")

Vector store setup complete


In [13]:

collection = client.collections.get("LangChain")

In [14]:
collection

In [15]:
response = collection.aggregate.over_all(total_count=True)
print(response)

AggregateReturn(properties={}, total_count=109)


In [16]:
response = collection.aggregate.over_all(group_by="source")
for group in response.groups:
    print(group.grouped_by.value, group.total_count)

netherlands-wikipedia-article-text.pdf 109


In [17]:
object = collection.query.fetch_objects(limit=1).objects[0]

In [18]:
object.properties.keys()

dict_keys(['text', 'page', 'source'])

In [19]:
print(object.properties.get("source"))
print(object.properties.get("page"))
print(object.properties.get("text"))

netherlands-wikipedia-article-text.pdf
6.0
Phytogeographically, the European Netherlands is shared between the Atlantic European and Central European provinces of the
Circumboreal Region within the Boreal Kingdom. According to the World Wide Fund for Nature, the European territory of the
Netherlands belongs to the ecoregion of Atlantic mixed forests. In 1871, the last old original natural woods were cut down, and most
woods today are planted monocultures of trees like Scots pine and trees that are not native to the Netherlands. These woods were
planted on anthropogenic heaths and sand-drifts (overgrazed heaths) (Veluwe). The Netherlands had a 2019 Forest Landscape
Integrity Index mean score of 0.6/10, ranking it 169th globally out of 172 countries.
Nitrogen pollution is a problem. The number of flying insects in the Netherlands has dropped by 75% since the 1990s.
Caribbean islands
In the Lesser Antilles islands of the Caribbean, the territories of Curaçao, Aruba and Sint Maarten have a

In [20]:
generateTask = "Qual é a comida tradicional deste país?. Answer in english"
# now generating the answer using Wikipedia
source_file = "netherlands-wikipedia-article-text.pdf"

In [22]:
db = WeaviateVectorStore.from_documents(docs, embeddings, client=client, index_name="LangChain")

In [23]:
print(db.similarity_search("Qual é a comida tradicional deste país?. Answer in english", k=3)[0].page_content)


moulded the country. In the course of the twentieth century this diet changed and became much more cosmopolitan, with most
global cuisines being represented in the major cities.
Modern culinary writers distinguish between three general regional forms of Dutch cuisine. The regions in the northeast of the
Netherlands, roughly the provinces of Groningen, Friesland, Drenthe, Overijssel and Gelderland north of the great rivers are the
least populated areas of the Netherlands. The late (18th century) introduction of large scale agriculture means that the cuisine is
generally known for its many kinds of meats. The relative lack of farms allowed for an abundance of game and husbandry, though
dishes near the coastal regions of Friesland, Groningen and the parts of Overijssel bordering the IJsselmeer also include a large
amount of fish. The various dried sausages, belonging to the metworst-family of Dutch sausages are found throughout this region.
Also smoked sausages are common, of which (


In [24]:
from langchain.prompts import ChatPromptTemplate

template="""You are helpful AI Assistant,perform the task that user ask or queries .Answer :
"""

In [37]:
prompt = """ You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [40]:

collection = client.collections.get("LangChain")

In [25]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
gemini_api_key = ""

In [26]:
model = genai.GenerativeModel('gemini-pro')

In [27]:
model = ChatGoogleGenerativeAI(
    model="gemini-pro", 
    google_api_key= gemini_api_key
)

In [28]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:

from langchain import HuggingFaceHub
huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
"""model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_kwargs={"temperature":1, "max_length":180}
)"""

<ipython-input-28-86bc09c35eb1>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(


In [30]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html

In [31]:
output_parser = StrOutputParser()

In [32]:
retriever = vectorstore.as_retriever()

In [33]:
retriever

VectorStoreRetriever(tags=['WeaviateVectorStore', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_weaviate.vectorstores.WeaviateVectorStore object at 0x000001B346320B80>, search_kwargs={})

In [34]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser

)

In [36]:
rag_chain.invoke("Explain the economy of Netherlands in 2020?")

'Sure, I can do that. Here are some examples of tasks that I can perform:\n\n* **Answer questions:** I can answer a wide range of questions on a variety of topics, including science, history, current events, and more.\n* **Provide information:** I can provide information on a variety of topics, such as weather, sports, stocks, and more.\n* **Translate languages:** I can translate text and speech between over 100 languages.\n* **Set alarms and timers:** I can set alarms and timers for you.\n* **Play games:** I can play a variety of games with you, such as trivia, hangman, and more.\n* **Tell jokes:** I can tell you jokes.\n* **Sing songs:** I can sing songs for you.\n* **Write stories:** I can write stories for you.\n* **Generate ideas:** I can generate ideas for a variety of topics, such as blog posts, essays, and more.\n* **Create content:** I can create content for a variety of purposes, such as social media posts, website articles, and more.\n* **Answer specific questions about prog